In [16]:
import pandas as pd

df = pd.read_csv('D:\python_projects\sentimentAnalysis\dipression_finder\dataset\depression_dataset_reddit_cleaned.csv')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # видалення пунктуації
    text = re.sub(r'[^\w\s]', '', text)

    # подання тексту у нижньому регістрі
    text = text.lower()

    # токенізація тексту
    tokens = nltk.word_tokenize(text)

    # видалення стоп-слів і слів корочше 3х символів
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]

    # видалення чисел
    tokens = [token for token in tokens if not token.isnumeric()]

    # лемматизація слів
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # об'єднання токенів
    text = ' '.join(tokens)

    return text

df['processed_text'] = df['clean_text'].apply(preprocess_text)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# утворення об'єкту TF-IDF векторизатора
vectorizer = TfidfVectorizer()
# векторизація входного тексту
X = vectorizer.fit_transform(df['processed_text'])


In [19]:
# визначення кількості кластерів
k = 5

# класстеризація за використанням алгоритму K-means
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(X)


In [ ]:
#  маркування 
df['cluster'] = clusters
df

In [ ]:
full_df = df.to_json()
with open('D:\python_projects\Dash\data\main_data.json', 'w') as f:
    f.write(full_df)

In [ ]:
# определение количества документов в клатере
doc_counts = df['cluster'].value_counts()

# преобразование в json и сохранение на диск
df_pie_diagram = doc_counts.to_json()
with open('D:\python_projects\Dash\my_series.json', 'w') as f:
    f.write(df_pie_diagram)
df_pie_diagram

In [ ]:
new_df=df[['processed_text','cluster']]
new_df

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def sentiment_score(new_df):
    sia = SentimentIntensityAnalyzer()
    for col in new_df.columns:
        if new_df[col].dtype == 'object':
            new_df['sentiment_score'] = new_df[col].apply(lambda x: sia.polarity_scores(x)['compound'])
    return new_df

sentiment_score(new_df)

In [ ]:
processed_sentiment_df=new_df.to_json()
with open('D:\python_projects\Dash\data\processed_sentiment_df.json', 'w') as f:
    f.write(processed_sentiment_df)

In [ ]:
from collections import Counter

def find_cluster_keywords(df):
    # утворення пустого датафрейму для ключових слів
    keywords_df = pd.DataFrame(columns=['cluster', 'keywords'])

    # Групування даних датафрейму за кластерами
    grouped = new_df.groupby('cluster')

    # Итерирування по групам
    for group_name, group_df in grouped:
        # утворення списку слів для кожного кластера
        words = []

        # ітерування по строках в середені групи
        for row in group_df.itertuples(index=False):
            # розбивка строк на окремі слова
            row_words = row.processed_text.split()

            # додавання кождного слова у список слів для потокового кластера
            words.extend(row_words)

        # підрахунок, частоті слів у списку
        word_counts = Counter(words)

        # Сортировка списку слів по кількості входжень в обратному порядку
        sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

        # утворення строки датафрейму з ключовими словами для кожного кластеру
        row = {'cluster': group_name, 'keywords': ', '.join([w[0] for w in sorted_words])}

        # додавання строки у датафрейм з ключовими словами
        keywords_df = keywords_df.append(row, ignore_index=True)

    # повернення датафрейму з ключовими словами
    return keywords_df
    

In [ ]:
keywords_df = find_cluster_keywords(df)

In [ ]:
keywords_df

In [ ]:
# определяем функцию для выбора первых 5 слов
def first_five_words(s):
    words = s.split()
    if len(words) >= 5:
        return ' '.join(words[:50])
    else:
        return ' '.join(words)

# применяем функцию к каждому элементу объекта Series
new_dataframe = pd.DataFrame(keywords_df['keywords'].apply(first_five_words))
new_dataframe
# преобразование в json и сохранение на диск
# df_gisto = new_dataframe.to_json()
# with open('D:\python_projects\Dash\data\joind_keywords.json', 'w') as f:
#     f.write(df_gisto)
    


In [ ]:
from sklearn.feature_extraction.text import Tfidfvectorizer
from sklearn.naive_bayes import MultinomialNB
from operator import itemgetter

def find_keywords(df, text_column, label_column, n_keywords=10):
    # утворення об'єкту TfidfVectorizer та навчання 
    vectorizer = TfidfVectorizer(stop_words="english')
    X = vectorizer.fit_transform(df[text_column])

# утворення об'єкту Наївного Байєса та навчання його нa векторах ознак і тегах класів
clf = Multinomialne()

clf.fit(x, df[label_column])

# отримано словник, де ключ - теги класів, a значення - ключових слів у класі

keywords = {}

feature_names = vectorizer.get_feature_names()

for i, class_label in enumerate(clf.classes_):
    top_keywords_idx = clf.coef_[i].argsort()[::-1][:n_keywords]
    top_keywords = itemgetter(*top_keywords_idx)(feature_names)
    keywords[class_label] = top_keywords

return keywords